In [1]:
import requests
import pandas as pd
import psycopg2
import os

# Función para traer API
def traer_api(url):
    try:
        respuesta_api = requests.get(url)

        # Corrobora si se pudo traer. Código de estado 200 es OK
        if respuesta_api.status_code == 200:
            # Si fue exitosa devolvemos los datos de la api en formato json
            return respuesta_api.json()
        else:
            # Si no fue exitosa se imprime el código de error
            print("Error al consultar la API:", respuesta_api.status_code)
            return None
    except Exception as exep:
        print("Error de conexión:", exep)
        return None

# URL API. Elegí la 7timer debido a que es sencillo de entender. Simplifiqué los datos que trae para que aún sea mas facil de comprender
url_api = "https://www.7timer.info/bin/api.pl?lon=-58.833&lat=-27.467&product=civillight&output=json"

# Aquí llamo a la función con la url seleccionada de 7timer
info_api = traer_api(url_api)

# Compruebo si se trajo datos de la api o sino devuelvo error
if info_api:
    # Solo muestro en pantalla
    print("Información de la API:", info_api)
    # Cargo en un DataFrame la información de la api normalizada en json, con la librería pandas, y luego lo muestro en pantalla
    df = pd.json_normalize(info_api, 'dataseries')
    print(df)

   # Acá comienzo con la concexión a Redshift. Pongo en variables de entorno los datos de conexión
    host = os.environ['REDSHIFT_HOST']
    port = os.environ['REDSHIFT_PORT']
    dbname = os.environ['REDSHIFT_DBNAME']
    user = os.environ['REDSHIFT_USER']
    password = os.environ['REDSHIFT_PASSWORD']

    # Establezco la conexión y creo un cursor
    conexion = psycopg2.connect(host=host, port=port, dbname=dbname, user=user, password=password)
    cur = conexion.cursor()

   # Con el cursor creado ejecuto el SQL para crear la tabla. Donde creo 3 columnas adicionales a los datos devueltos por la API. Un identificador primario, una constante de la ciudad elegida para el clima y un Timestamp que nos da la fecha y hora actual cuando se inserta el dato
    cur.execute("""
    CREATE TABLE IF NOT EXISTS Javier_Prueba_API (
        id INTEGER PRIMARY KEY IDENTITY(1,1),
        ciudad_actual VARCHAR(100) DEFAULT 'CIUDAD DE CORRIENTES',
        fecha_hora TIMESTAMP DEFAULT GETDATE(),
        date INT,
        weather VARCHAR(20),
        max_temp INT,
        min_temp INT,
        wind_max INT
    )
    """)

    # Inserto los datos del dataframe en la tabla
    for index, row in df.iterrows():
        cur.execute("INSERT INTO Javier_Prueba_API (date, weather, max_temp, min_temp, wind_max) VALUES (%s, %s, %s, %s, %s)", 
                    (row['date'], row['weather'], row['temp2m.max'], row['temp2m.min'], row['wind10m_max']))
        
    conexion.commit()

    # Y cierro la conexión
    cur.close()
    conexion.close()
else:
    print("Hubo problemas para obtener los datos de la API.")

Información de la API: {'product': 'civillight', 'init': '2024040106', 'dataseries': [{'date': 20240401, 'weather': 'clear', 'temp2m': {'max': 35, 'min': 24}, 'wind10m_max': 3}, {'date': 20240402, 'weather': 'lightrain', 'temp2m': {'max': 27, 'min': 20}, 'wind10m_max': 3}, {'date': 20240403, 'weather': 'cloudy', 'temp2m': {'max': 36, 'min': 23}, 'wind10m_max': 3}, {'date': 20240404, 'weather': 'ishower', 'temp2m': {'max': 32, 'min': 24}, 'wind10m_max': 3}, {'date': 20240405, 'weather': 'lightrain', 'temp2m': {'max': 26, 'min': 23}, 'wind10m_max': 3}, {'date': 20240406, 'weather': 'cloudy', 'temp2m': {'max': 28, 'min': 21}, 'wind10m_max': 3}, {'date': 20240407, 'weather': 'cloudy', 'temp2m': {'max': 31, 'min': 22}, 'wind10m_max': 2}]}
       date    weather  wind10m_max  temp2m.max  temp2m.min
0  20240401      clear            3          35          24
1  20240402  lightrain            3          27          20
2  20240403     cloudy            3          36          23
3  20240404    i